## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Libraries</p>

In [1]:
import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import urllib.request as req
import json
from tqdm import tqdm, tqdm_notebook
import requests

import pandas as pd
import re
import string

from pandas.io.formats.style import Styler
import seaborn as sns

from itertools import chain

pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

tqdm.pandas()

rc = {
    "axes.facecolor": "#F8F8F8", 
    "figure.facecolor": "#F8F8F8", 
    "axes.edgecolor": "#000000",  
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000", 
    "ytick.color": "#000000",
    "grid.alpha": 0.4 
}

sns.set(rc=rc) 
palette = ['#ff7f50', '#ffd700', '#ffdab9', '#9fe2bf',
           '#d2b48c', '#008080', '#98ff98', '#000080']


from colorama import Style, Fore 
blk = Style.BRIGHT + Fore.BLACK
gld = Style.BRIGHT + Fore.YELLOW
grn = Style.BRIGHT + Fore.GREEN
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

In [2]:
def magnify(is_test : bool = False): 
    base_color = '#b57edc'
    if is_test:
        highlight_target_row = []
    else:
        highlight_target_row = dict(selector = 'tr:last-child',
                            props = [('background-color', f'{base_color}' + '20')]) 
    
    return [dict(selector="th", 
                props=[("font-size", "11pt"),
                    ('background-color', f'{base_color}'),
                    ('color', 'white'),
                    ('font-weight', 'bold'),
                    ('border-bottom', '0.1px solid white'), 
                    ('border-left', '0.1px solid white'), 
                    ('text-align', 'right')]),
        
            dict(selector='th.blank.level0', 
                props=[('font-weight', 'bold'),
                        ('border-left', '1.7px solid white'),
                        ('background-color', 'white')]),

            dict(selector="td", 
                    props=[('padding', "0.5em 1em"), 
                        ('text-align', 'right')]),

            dict(selector="th:hover",
                    props=[("font-size", "14pt")]),

            dict(selector="tr:hover td:hover",
                    props=[('max-width', '250px'),
                        ('font-size', '14pt'),
                        ('color', f'{base_color}'),
                        ('font-weight', 'bold'),
                        ('background-color', 'white'),
                        ('border', f'1px dashed {base_color}')]),
            
            dict(selector="caption", 
                props=[(('caption-side', 'bottom'))])] + highlight_target_row

def stylize_simple(df: pd.DataFrame, caption: str) -> Styler:
    """
        Args:
            df: any dataframe (train/test/origin)

        Returns:
            s: the dataframe wrapped into Styler.
    """
    s = df
    s = s.style.set_table_styles(magnify(True)).set_caption(f"{caption}")
    return s

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Intro</p>

Collecting Tokyo Real Estate Case Data

 <a href = 'http://www.oshimaland.co.jp'> 오시마랜드(Oshimaland)</a>
>This website showcases buildings with unfortunate past incidents such as suicide cases, corpse disposal incidents, and murder cases.


## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Data Crawling</p>

In [3]:
jp_district_list = [
    '新宿区', '台東区', '墨田区', '豊島区', '渋谷区',
    '港区', '世田谷区', '大田区', '中野区', '中央区',
    '北区', '葛飾区', '杉並区', '江東区', '江戸川区',
    '文京区', '荒川区', '板橋区', '品川区', '千代田区',
    '足立区', '目黒区', '練馬区', '府中市', '八王子市',
    '日野市', '国分寺市', '町田市', '青梅市', '調布市',
    '武蔵野市', 'あきる野市', '三鷹市', '小金井市', '東村山市',
    '多摩市', '国立市', '狛江市', '西東京市', '立川市',
    '小平市', '羽村市', '武蔵村山市', '奥多摩町', '昭島市',
    '福生市'
]

In [4]:
def getJsonData(data):
    json_url = 'https://api.oshimaland.co.jp/map'
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Origin": "https://www.oshimaland.co.jp"
    }
    response = requests.post(json_url, headers=headers, json=data)
    json_data = response.json()
    return json_data


In [5]:
def dataList(data, json_data):
    jsonValueList = []
    origin = list(data.values())
    flatten_list = list(chain(*origin))
    for idx in range(len(flatten_list)):
        for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
            jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])
    return jsonValueList

In [6]:
def getinfo(jsonValueList):
    getDataList = []
    for idx in tqdm_notebook(range(len(jsonValueList))):
        json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
        url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
        code = req.urlopen(url)
        soup = BeautifulSoup(code, 'html.parser')
        json_data = json.loads(soup.text)
        getDataList.append({
            'info' : json_data['info'],
            'address' : json_data['ad'],
            'dt' : json_data['dt'],
            'cr' : json_data['cr'],
        })
        
    return pd.DataFrame(getDataList)

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#0B666A; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #0B666A">각 구역 마다, json 파라미터 값이 모두 다르므로, 파라미터 값을 변경하면서 데이터를 가져와야 한다.</p>
network tap -> fetch/xhr -> map.json

In [7]:
#* '新宿区'
data = {'keys': ["1330021123012133", "1330021123012311", "1330021123013022", "1330021123013200"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df1 = getinfo(jsonValueList)
display(stylize_simple(df1.head(4), '新宿区'))

  0%|          | 0/135 [00:00<?, ?it/s]

,info,address,dt,cr
0,819号室で女性死亡,東京都新宿区歌舞伎町一丁目20-2,平成30年5月23日,平成30年7月4日
1,飛び降り自殺,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,令和3年5月18日,令和3年5月26日
2,告知事項あり ※管理会社に直接確認済み ハイム大成ビル 403号室 賃料¥78.000- 管理費¥2.000- 前入居者が室内で自殺,東京都新宿区西新宿七丁目18-13,1年以内,令和3年2月23日
3,１４Ｆから女性が飛び降り自殺,東京都新宿区西新宿七丁目17-13パレステュディオ新宿WEST,令和元年9月23日,令和元年9月23日


In [8]:
#* '台東区'
data = {'keys': ["1330021123100332", "1330021123102110"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df2 = getinfo(jsonValueList)
display(stylize_simple(df2.head(4), '台東区'))

  0%|          | 0/23 [00:00<?, ?it/s]

,info,address,dt,cr
0,火災による死亡,東京都台東区東上野六丁目8-1鍋谷ビル1階,令和2年1月5日,令和2年1月6日
1,Log上野駅前903号室病死（死後時間経過）,東京都台東区東上野四丁目10-12 Log上野駅前,2018年夏,令和元年8月24日
2,"5階 西向き2LDK（LDK14.7帖、洋室6.3帖、洋室5.3帖、洋室4.5帖） 専有面積74.15㎡ バルコニー面積4.97㎡ 価格6,480万円 管理費等14,760円 修繕積立金14,910円 告知事項あり",東京都台東区東上野五丁目21-5 ジェイパーク上野アクシス,令和4年8月4日,令和4年8月5日
3,白骨死体,東京都台東区東上野六丁目12-4,平成29年5月19日,平成29年5月19日


In [9]:
#* '墨田区'
data = {'keys': ["1330021123101223", "1330021123101232", "1330021123103001", "1330021123103010"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df3 = getinfo(jsonValueList)
display(stylize_simple(df3.head(4), '墨田区'))

  0%|          | 0/17 [00:00<?, ?it/s]

,info,address,dt,cr
0,共用部外階段より投身自殺,東京都台東区花川戸一丁目13-6,2008年以前,令和2年7月9日
1,死体発見,東京都台東区浅草二丁目33-1スーパーホテル浅草703,平成25年12月7日,平成26年5月20日
2,首吊り自殺,東京都台東区花川戸一丁目3-6 花川戸ビル,平成19年1月27日,令和4年10月21日
3,修繕工事時足場首吊り自殺 1Fにて,東京都台東区雷門二丁目17-8,2007年,平成29年10月11日


In [10]:
#* '豊島区'
data = {'keys': ["1330021123011210"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df4 = getinfo(jsonValueList)
display(stylize_simple(df4.head(4), '豊島区'))

  0%|          | 0/8 [00:00<?, ?it/s]

,info,address,dt,cr
0,心理的瑕疵物件,東京都豊島区南池袋二丁目30-16,令和3年11月,令和4年5月16日
1,11階あたりの部屋から飛び降り自殺 1階の裏口に落下,東京都豊島区南池袋二丁目9-17,2013年1月ごろ,平成28年10月8日
2,男性の変死体発見,東京都豊島区南池袋二丁目9-17ニッシンパレステージ池袋804,2018年６月,平成30年10月22日
3,飛び降り自殺,東京都豊島区南池袋二丁目8-18,平成27年8月30日,平成28年9月9日


In [11]:
#* '渋谷区'
data = {'keys': ["1330021123030131"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df5 = getinfo(jsonValueList)
display(stylize_simple(df5.head(4), '渋谷区'))

  0%|          | 0/4 [00:00<?, ?it/s]

,info,address,dt,cr
0,死体発見,東京都渋谷区神南一丁目4-17,平成22年2月24日,令和4年9月18日
1,飛び降り自殺,東京都渋谷区宇田川町10-3,平成27年8月21日,平成27年8月22日
2,転落死,東京都渋谷区神南一丁目8-18クオリア神南フラッツ建設現場11階↓,平成19年8月5日,
3,10階非常階段から落下して死亡,東京都渋谷区神南一丁目20-13,2021年前半,令和4年9月6日


In [12]:
#* '港区'
data = {'keys': ["1330020303012333"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df6 = getinfo(jsonValueList)
display(stylize_simple(df6.head(4), '港区'))

  0%|          | 0/6 [00:00<?, ?it/s]

,info,address,dt,cr
0,死体発見,愛知県名古屋市港区港陽一丁目1-50ファミール港陽ベイレジデンス9階東端の部屋,平成30年10月29日,平成30年10月30日
1,自殺による転落死,愛知県名古屋市港区港楽一丁目1−１０ 市営港楽荘1棟,平成26年5月12日,平成26年5月12日
2,事故住宅,愛知県名古屋市港区港楽一丁目1-10 市営港楽荘1棟 201,不明,平成29年2月22日
3,2名の死体発見,愛知県名古屋市港区港楽一丁目1-10 市営港楽荘1棟,令和4年11月3日,令和4年11月4日


In [13]:
#* '世田谷区'
data = {'keys': ["1330021123021313", "1330021123021330", "1330021123021331"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df7 = getinfo(jsonValueList)
display(stylize_simple(df7.head(4), '世田谷区'))

  0%|          | 0/11 [00:00<?, ?it/s]

,info,address,dt,cr
0,孤高死,東京都世田谷区梅丘三丁目12-5メゾン吉田,平成23年5月頃,平成25年6月2日
1,３０７号室の居室にて自殺,東京都世田谷区若林四丁目39-4,2011/8月,平成23年12月2日
2,腐乱死。 体液が玄関扉からも溢れ、一帯が強烈な臭いでした。,東京都世田谷区若林五丁目18-8 エステート増永 201号室,令和3年7月,令和4年9月10日
3,火災による死亡,東京都世田谷区梅丘三丁目13-7,平成23年1月3日,


In [14]:
#* '大田区'
data = {'keys': ["1330021123213012"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df8 = getinfo(jsonValueList)
display(stylize_simple(df8.head(4), '大田区'))

  0%|          | 0/10 [00:00<?, ?it/s]

,info,address,dt,cr
0,自殺,東京都大田区西蒲田七丁目2-3第2醍醐ビル6F,平成22年10月14日,
1,自殺か病死かは分からないがこのホテルの5F512号室で人が亡くなっていたらしい。朝起きたらちょうど警察が来ていて遺体を運び出そうとしていた場面に遭遇した。,東京都大田区蒲田五丁目18-4,令和5年2月26日,令和5年2月26日
2,ガス自殺,東京都大田区蒲田五丁目11-9 404,平成24年7月28日,平成24年7月28日
3,駐車場 死体発見,東京都大田区西蒲田七丁目68 グランデュオ蒲田,平成21年5月15日,令和4年10月14日


In [15]:
#* '中野区'
data = {'keys': ["1330021123012000", "1330021123012001", "1330021123012002", "1330021123012003"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df9 = getinfo(jsonValueList)
display(stylize_simple(df9.head(4), '中野区'))

  0%|          | 0/28 [00:00<?, ?it/s]

,info,address,dt,cr
0,住人ではなかった女の子が7階から駐車場へ飛び降り。,東京都中野区新井二丁目31-1,2000年1月,平成27年9月24日
1,自殺,東京都中野区野方一丁目29-1アーバンアメニティ中野5階,2015年ごろ,平成30年7月14日
2,本籍・住所・氏名・年齢不詳、男性、身長160センチメートル、中肉 上記の者は令和2年7月28日午後10時35分頃、東京都中野区野方1丁目5番17号中野ダイヤハイツ309において死亡の状態で発見されました。遺体は火葬に付し遺骨は保管してあります。心当たりの方は、当区健康福祉部生活援護課までお申し出ください。（官報号外第170号2021/7/26）,東京都中野区野方一丁目5-17,令和2年7月28日,令和3年8月11日
3,2007年に513号室の室内で旦那さんが介護疲れで首つり自殺,東京都中野区中野五丁目24-16中野第二コーポ513号室,2007年,平成30年8月4日


In [16]:
#* '中央区'
data = {'keys': ["1330020220302013", "1330020220302031", "1330020220302102", "1330020220302120"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df10 = getinfo(jsonValueList)
display(stylize_simple(df10.head(4), '中央区'))

  0%|          | 0/44 [00:00<?, ?it/s]

,info,address,dt,cr
0,スーパーホテル709号室にて死体発見,兵庫県神戸市中央区山本通二丁目1-11,平成19年7月26日,平成25年2月5日
1,元従業員が飛び降り自殺,兵庫県神戸市中央区加納町四丁目8-15,2015年頃,平成29年11月1日
2,首吊り自殺,兵庫県神戸市中央区二宮町四丁目8-10 KDXレジデンス三宮自転車置き場,～5年前,平成27年3月14日
3,絞殺,兵庫県神戸市中央区二宮町二丁目13-23,平成30年9月15日,平成30年9月16日


In [17]:
#* '北区'
data = {'keys': ["1330020223011330", "1330020223011332"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df11 = getinfo(jsonValueList)
display(stylize_simple(df11.head(4), '北区'))

  0%|          | 0/5 [00:00<?, ?it/s]

,info,address,dt,cr
0,３階 男性死亡,大阪府堺市北区新金岡町四丁目2-8,令和4年4月6日,令和4年4月6日
1,看護士から虐待を受けた老人が死亡,大阪府堺市北区新金岡町5丁1-3病院内,平成21年9月15日,令和元年7月27日
2,飛び降り自殺 ２回目,大阪府堺市北区新金岡町四丁目5-1,令和5年3月8日,令和5年3月8日
3,6号棟503号室。心理的瑕疵有。,大阪府堺市北区金岡町704,2022年3月頃,令和4年8月20日


In [18]:
#* '葛飾区'
data = {'keys': ["1330021123110010", "1330021123110012"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df12 = getinfo(jsonValueList)
display(stylize_simple(df12.head(4), '葛飾区'))

  0%|          | 0/5 [00:00<?, ?it/s]

,info,address,dt,cr
0,告知事項あり,東京都葛飾区立石五丁目23-12,2021/04/26,令和3年7月4日
1,コーポ山元102号室｡告知事項あり https://www.homes.co.jp/chintai/b-37011320094900/,東京都葛飾区立石三丁目30-9,不明,平成29年5月19日
2,2階 北向きワンルーム（洋室6.5帖） 専有面積17.36㎡ 賃料5万円 告知事項あり,東京都葛飾区立石三丁目27-18 グリュックスハイム,令和4年8月5日,令和4年8月5日
3,火災による死亡,東京都葛飾区立石七丁目6,令和4年3月18日,令和4年3月19日


In [19]:
#* '杉並区'
data = {'keys': ["1330021123003031", "1330021123003120"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df13 = getinfo(jsonValueList)
display(stylize_simple(df13.head(4), '杉並区'))

  0%|          | 0/15 [00:00<?, ?it/s]

,info,address,dt,cr
0,一階にて独居男性の硫化水素による自殺未遂発生,東京都杉並区阿佐谷南三丁目7-7,2011-2012？,令和元年9月17日
1,独居の老人が孤高死,東京都杉並区阿佐谷南三丁目6-20第一赤松荘,2016年頃,令和2年3月23日
2,404号室 角部屋 告知事項有 2022年8月室内スケルトンリフォーム済,東京都杉並区成田東五丁目34-14 住友南阿佐ヶ谷シティハウス,令和4年9月9日,令和4年9月10日
3,火災による死亡,東京都杉並区成田東五丁目30-22,平成26年5月22日,平成26年5月22日


In [20]:
#* '文京区'
data = {'keys': ["1330021123102000", "1330021123102001", "1330021123102002", "1330021123102003"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df14 = getinfo(jsonValueList)
display(stylize_simple(df14.head(4), '文京区'))

  0%|          | 0/16 [00:00<?, ?it/s]

,info,address,dt,cr
0,孤高死,東京都文京区小石川二丁目8-13,15年前,令和2年1月18日
1,コア伝通院 3階 告知事項あり。 https://suumo.jp/chintai/tokyo/sc_bunkyo/jnc_000027501988/?bc=100091752252,東京都文京区春日一丁目11-13,平成29年5月15日,平成29年5月14日
2,5年ほど前ラーメン屋の前の店舗とんかつ屋主店の中で 首つり,東京都文京区小石川一丁目15-9,5ねんほどまえ,平成30年7月7日
3,無理心中,東京都文京区本郷五丁目11-4梨の木坂ハウス,平成30年9月26日,平成30年9月26日


In [21]:
#* '荒川区'
data = {'keys': ["1330021123100112", "1330021123100130"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df15 = getinfo(jsonValueList)
display(stylize_simple(df15.head(4), '荒川区'))

  0%|          | 0/16 [00:00<?, ?it/s]

,info,address,dt,cr
0,2階 南東向き 20.95㎡ 告知事項アリ,東京都荒川区荒川七丁目34-7,不明,平成28年3月25日
1,"2階 告知事項有 13階, 6階 共用部告知事項あり",東京都荒川区荒川七丁目36-3,----,令和5年6月7日
2,マークスタワー24階より飛び降り自殺,東京都荒川区荒川七丁目46-1,平成25年11月22日,平成25年11月28日
3,火災による死亡,東京都荒川区荒川七丁目24-3,平成20年7月4日,


In [22]:
#* '板橋区'
data = {'keys': ["1330021121233221", "1330021121233223"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df16 = getinfo(jsonValueList)
display(stylize_simple(df16.head(4), '板橋区'))

  0%|          | 0/35 [00:00<?, ?it/s]

,info,address,dt,cr
0,孤高死,東京都板橋区仲宿34-8,2015,平成29年7月16日
1,心理的瑕疵物件,東京都板橋区仲宿36-13,2013,平成29年8月14日
2,4階 角部屋 告知事項有,東京都板橋区仲宿57-3 メゾン・ド・セレール,令和4年10月13日,令和4年10月13日
3,孤高死,東京都板橋区仲宿35-7,2014,平成29年7月16日


In [23]:
#* '品川区'
data = {'keys': ["1330021123033320", "1330021123033321", "1330021123033322", "1330021123033323"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df17 = getinfo(jsonValueList)
display(stylize_simple(df17.head(4), '品川区'))

  0%|          | 0/20 [00:00<?, ?it/s]

,info,address,dt,cr
0,高所作業車が倒れて死亡,東京都品川区二葉一丁目1建設現場,平成25年1月16日,平成25年1月17日
1,"3階 東向きワンルーム 専有面積13.94㎡ 賃料4.7万円 管理費等3,000円 告知事項あり",東京都品川区豊町二丁目18-2 メゾン・ド・リュビ,令和4年7月2日,令和4年7月2日
2,火災による死亡,東京都品川区西品川一丁目11-10青雲荘207&210,平成18年8月22日,
3,販売物件 告知事項アリ 記載,東京都品川区豊町二丁目14-2,2022/5完成販売物件,令和4年7月3日


In [24]:
#* '千代田区'
data = {'keys': ["1330021123102023"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df18 = getinfo(jsonValueList)
display(stylize_simple(df18.head(4), '千代田区'))

  0%|          | 0/4 [00:00<?, ?it/s]

,info,address,dt,cr
0,飛び降り自殺,東京都千代田区九段南一丁目3-1,2021年05月初旬,令和3年6月1日
1,敷地内 死体発見,東京都千代田区神田神保町二丁目4-29 松本ビル,平成19年8月21日,令和4年10月20日
2,自殺,東京都千代田区神田神保町三丁目11-1 6階,2011年,令和3年10月19日
3,メゾン・ド・ヴィレ神田神保町406号室 告知事項有り,東京都千代田区神田神保町二丁目六丁目11-4,不明,平成28年9月24日


In [25]:
#* '足立区'
data = {'keys': ["1330021121323012", "1330021121323030"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df19 = getinfo(jsonValueList)
display(stylize_simple(df19.head(4), '足立区'))

  0%|          | 0/11 [00:00<?, ?it/s]

,info,address,dt,cr
0,201号室 告知事項有,東京都足立区中央本町四丁目6-2 セントラルハイツ,令和4年4月27日,令和4年6月7日
1,102号室 告知事項有,東京都足立区中央本町四丁目6-2 セントラルハイツ,令和4年5月10日,令和4年5月11日
2,告知事項あり（不動産情報より）,東京都足立区中央本町四丁目6-2 セントラルハイツ 1階,2020年3月現在,令和2年3月25日
3,604号室から女性が犬を抱き飛び降り。,東京都足立区中央本町一丁目18-12,2011年頃,平成28年8月23日


In [26]:
#* '目黒区'
data = {'keys': ["1330021123030333"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df20 = getinfo(jsonValueList)
display(stylize_simple(df20.head(4), '目黒区'))

  0%|          | 0/6 [00:00<?, ?it/s]

,info,address,dt,cr
0,心理的瑕疵,東京都目黒区上目黒二丁目17-20,2012,令和4年3月15日
1,練炭自殺,東京都目黒区上目黒二丁目16-11,2010〜2011,平成30年11月21日
2,賃貸物件サイトに病死の部屋がある旨、記載あり,東京都目黒区上目黒二丁目18-2,2014年？,平成26年10月6日
3,訂正して下さり、ありがとうございます。 東京都目黒区上目黒2-15-4 マスダビル 増田隆良 30.10.24.,東京都目黒区上目黒二丁目15-4,平成30年10月24日,平成30年10月23日


In [27]:
#* '練馬区'
data = {'keys': ["1330021123001130"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df21 = getinfo(jsonValueList)
display(stylize_simple(df21.head(4), '練馬区'))

  0%|          | 0/18 [00:00<?, ?it/s]

,info,address,dt,cr
0,8階から飛び降り自殺,東京都練馬区練馬三丁目1-2 ドゥーエ練馬Ⅱ,平成28年9月15日,平成28年9月15日
1,飛び降り自殺,東京都練馬区練馬三丁目2-8ライオンズマンション練馬14階↓,平成21年6月16日,
2,刺殺,東京都練馬区中村北一丁目11-3フォーラムイン東京718,平成20年4月3日,
3,告知事項有,東京都練馬区練馬一丁目6-8 ボナール縁 5階/角部屋,不明,平成29年11月1日


In [28]:
#* '府中市'
data = {'keys': ["1330021122121111", "1330021122121113"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df22 = getinfo(jsonValueList)
display(stylize_simple(df22.head(4), '府中市'))

  0%|          | 0/6 [00:00<?, ?it/s]

,info,address,dt,cr
0,刺殺,東京都府中市宮西町一丁目5,平成19年2月14日,令和3年10月16日
1,心理的瑕疵物件,東京都府中市府中町二丁目10-1,平成30年4月11日,平成30年5月30日
2,ホテルコンチネンタル府中、地下１階 自己啓発主催者ら書類送検＝セミナー中に男性死亡―遺棄致死容疑で・警視庁,東京都府中市府中町一丁目5-1,平成22年1月27日,平成27年8月17日
3,病死,東京都府中市本町一丁目13-1,平成18年2月18日,平成28年12月21日


In [29]:
#* '八王子市'
data = {'keys': ["1330021122030003", "1330021122030021"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df23 = getinfo(jsonValueList)
display(stylize_simple(df23.head(4), '八王子市'))

  0%|          | 0/7 [00:00<?, ?it/s]

,info,address,dt,cr
0,首吊り自殺,東京都八王子市元本郷町三丁目24-1,首吊り自殺,平成30年12月8日
1,老人孤高死,東京都八王子市元本郷町四丁目14-9,令和3年2月8日,令和3年2月19日
2,105号室 孤高死・腐乱,東京都八王子市元本郷町三丁目7-5,2017年7～8月頃,令和元年7月27日
3,火災による死亡,東京都八王子市元本郷町三丁目9-16,平成28年3月17日,平成28年3月17日


In [30]:
#* '日野市'
data = {'keys': ["1330021122120000"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df24 = getinfo(jsonValueList)
display(stylize_simple(df24.head(4), '日野市'))

  0%|          | 0/1 [00:00<?, ?it/s]

,info,address,dt,cr
0,202号室 角部屋 告知事項有,東京都日野市神明四丁目7-12 コートピア日野,令和4年10月8日,令和4年10月9日


In [31]:
#* '国分寺市'
data = {'keys': ["1330021122103100"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df25 = getinfo(jsonValueList)
display(stylize_simple(df25.head(4), '国分寺市'))

  0%|          | 0/7 [00:00<?, ?it/s]

,info,address,dt,cr
0,首吊り自殺,東京都国分寺市戸倉一丁目8-25,2021年5月,令和3年6月27日
1,火災により78歳男性が4日後に亡くなった,東京都国分寺市日吉町四丁目6-1,平成29年7月22日,平成30年1月24日
2,5階 角部屋 告知事項有り,東京都国分寺市戸倉一丁目8-25 PlaceK,令和4年10月6日,令和4年10月7日
3,6階 角部屋 告知事項有り,東京都国分寺市戸倉一丁目8-25 PlaceK,令和4年10月6日,令和4年10月7日


In [32]:
#* '町田市'
data = {'keys': ["1330021122302311", "1330021122302313", "1330021122303200", "1330021122303202"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df26 = getinfo(jsonValueList)
display(stylize_simple(df26.head(4), '町田市'))

  0%|          | 0/11 [00:00<?, ?it/s]

,info,address,dt,cr
0,絞殺,東京都町田市森野二丁目4-8 CASA VICENS,平成19年5月6日,平成25年5月12日
1,心中,神奈川県相模原市南区鵜野森二丁目27-6鵜野森団地F棟402,平成28年2月18日,平成28年2月19日
2,精神的瑕疵,東京都町田市森野五丁目15-2,2017,平成30年3月12日
3,告知事項あり,東京都町田市森野二丁目3-4 ビラージュシンワ 301号室,不明,平成31年3月18日


In [33]:
#* '青梅市'
data = {'keys': ["1330021120221230", "1330021120221232"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df27 = getinfo(jsonValueList)
display(stylize_simple(df27.head(4), '青梅市'))

  0%|          | 0/8 [00:00<?, ?it/s]

,info,address,dt,cr
0,熱中症により孤高死,東京都青梅市東青梅二丁目16-12,2014年か2015年の8月頃,平成31年1月8日
1,夕方頃に救急車や警察が来ていた。防護服着ていた人もいたから、多分そういうことがあったと思われる。,東京都青梅市東青梅三丁目26-4,令和3年7月後半,令和4年7月29日
2,心理的瑕疵有,東京都青梅市東青梅二丁目19-13 ホーユウコンフォルト東青梅 3階,時期不明,平成30年9月1日
3,孤高死もしくは自然死,東京都青梅市東青梅三丁目九丁目5-206,2010年代,令和2年8月30日


In [34]:
#* '調布市'
data = {'keys': ["1330021122131212"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df28 = getinfo(jsonValueList)
display(stylize_simple(df28.head(4), '調布市'))

  0%|          | 0/5 [00:00<?, ?it/s]

,info,address,dt,cr
0,告知事項あり（不動産情報より）,東京都調布市小島町二丁目12-1 ＡＬＰＩＮＥ ＣＯＵＲＴ １ 1階,2020年1月現在,令和2年1月24日
1,傷害致死,東京都調布市小島町二丁目14-26ユマニテ調布103,平成24年9月18日～平成24年9月20日,平成24年9月24日
2,ファミール小島町2F（部屋番号不明） 心理的瑕疵物件あり,東京都調布市小島町二丁目19-5,不明,平成26年4月24日
3,◆エスター調布B203号室◆ 死亡している70代くらいの女性と50代くらいの男性発見。外傷なし。連絡が取れず施錠されていたので、消防がベランダの窓を割って部屋に入り発見。いずれもあおむけの状態で倒れていた。 https://www.tokyo-np.co.jp/article/150176 https://www.sankei.com/article/20211221-JCD34JMV4FJB5KUT54ME6VEQEY/,東京都調布市小島町二丁目12,令和4年1月20日,令和4年2月15日


In [35]:
#* '三鷹市'
data = {'keys': ["1330021122113303", "1330021122113312"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df29 = getinfo(jsonValueList)
display(stylize_simple(df29.head(4), '三鷹市'))

  0%|          | 0/5 [00:00<?, ?it/s]

,info,address,dt,cr
0,29歳女性が首吊り自殺,東京都三鷹市上連雀九丁目14-14,2018年3月9日,平成30年3月16日
1,心理的瑕疵物件 不動産情報に記載あり,東京都三鷹市上連雀八丁目2-16 コスモハイツ 2階,時期不明,平成29年11月8日
2,心理的瑕疵有 2019年02月：内装リフォーム済,東京都三鷹市下連雀九丁目7-12 カサーレ三鷹 3階/南向き住戸,時期不明,令和元年5月5日
3,告知事項が御座います,東京都三鷹市下連雀九丁目5-40 アケボノマンション 202号室,不明,平成27年8月8日


In [36]:
#* '小金井市'
data = {'keys': ["1330021122112031"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df30 = getinfo(jsonValueList)
display(stylize_simple(df30.head(4), '小金井市'))

  0%|          | 0/1 [00:00<?, ?it/s]

,info,address,dt,cr
0,シャトー小金井10階 火災で1人死亡,東京都小金井市本町六丁目5-3,平成15年5月10日,平成26年3月21日


In [37]:
#* '東村山市'
data = {'keys': ["1330021120323303", "1330021120323321"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df31 = getinfo(jsonValueList)
display(stylize_simple(df31.head(4), '東村山市'))

  0%|          | 0/15 [00:00<?, ?it/s]

,info,address,dt,cr
0,告知事項あり（不動産情報より）,東京都東村山市本町二丁目20-40 グレイスマンション 2階,2020年1月現在,令和2年1月24日
1,心理的瑕疵有,東京都東村山市本町一丁目21 エヴァース 1階/102号室,時期不明,平成30年9月1日
2,火災による死亡,東京都東村山市本町一丁目17-28,平成29年1月3日,平成29年1月3日
3,告知事項あり,東京都東村山市本町四丁目2-3,令和2年12月25日,令和3年1月5日


In [38]:
#* '多摩市'
data = {'keys': ["1330021122123001"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df32 = getinfo(jsonValueList)
display(stylize_simple(df32.head(4), '多摩市'))

  0%|          | 0/3 [00:00<?, ?it/s]

,info,address,dt,cr
0,208号 64.40㎡ 角部屋 告知事項あり,東京都多摩市貝取1493,不明,平成27年2月5日
1,火災による死亡,東京都多摩市貝取1707,平成25年3月29日,平成25年3月30日
2,心理的瑕疵,東京都多摩市貝取六丁目10-10,2015年頃,平成30年4月13日


In [39]:
#* '国立市'
data = {'keys': ["1330021122103202"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df33 = getinfo(jsonValueList)
display(stylize_simple(df33.head(4), '国立市'))

  0%|          | 0/4 [00:00<?, ?it/s]

,info,address,dt,cr
0,死体発見,東京都国立市富士見台二丁目46 UR国立富士見台第2団地2-5号棟506,平成27年9月22日,平成27年9月23日
1,友人が自殺しました。,東京都国立市富士見台二丁目46-4 第2団地5-110,平成28年7月10日,平成30年3月10日
2,遺体発見,東京都国立市富士見台二丁目四六丁目2-5-401,2020年6月１７日,令和2年7月7日
3,2階上がって左端の部屋、異臭により通報、浴室で男性の腐乱死体発見。体は溶け下半身は白骨化していた模様。男性はパニック障害者で孤高死でした。,東京都国立市富士見台二丁目43-2,2019年秋,令和2年7月12日


In [40]:
#* '狛江市'
data = {'keys': ["1330021123022001", "1330021123022003"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df34 = getinfo(jsonValueList)
display(stylize_simple(df34.head(4), '狛江市'))

  0%|          | 0/9 [00:00<?, ?it/s]

,info,address,dt,cr
0,心理的瑕疵有 内装リフォーム済：2018年01月,東京都狛江市和泉本町一丁目36-3 狛江セントラルハイツ３号棟 8階,時期不明,平成30年5月22日
1,205号室 過労死,東京都狛江市岩戸北一丁目12-10,平成30年3月25日,令和元年10月8日
2,遺体発見,東京都狛江市和泉本町一丁目13-16,平成30年1月28日,平成30年1月29日
3,15階より 飛び降り自殺,東京都狛江市和泉本町一丁目36-1 狛江セントラルハイツ1号棟,平成20年6月21日,令和4年9月26日


In [41]:
#* '西東京市'
data = {'keys': ["1330021122111210", "1330021122111212"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df35 = getinfo(jsonValueList)
display(stylize_simple(df35.head(4), '西東京市'))

  0%|          | 0/7 [00:00<?, ?it/s]

,info,address,dt,cr
0,男性遺体発見,東京都西東京市南町三丁目3-12,平成19年2月2日,平成26年1月23日
1,シェア田無 告知事項あり,東京都西東京市田無町二丁目12-21,2018,令和3年4月26日
2,告知事項有,東京都西東京市田無町四丁目26-5,平成19年11月14日,令和元年12月3日
3,心理的瑕疵あり,東京都西東京市南町五丁目9-12,2018年11月ごろ,令和元年9月6日


In [42]:
#* '立川市'
data = {'keys': ["1330021122100232"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df36 = getinfo(jsonValueList)
display(stylize_simple(df36.head(4), '立川市'))

  0%|          | 0/1 [00:00<?, ?it/s]

,info,address,dt,cr
0,薬使用後に７階位の踊り場から飛び降り自殺。 １８歳位の男性,東京都立川市泉町1156-4,８年位前,平成26年8月23日


In [43]:
#* '小平市'
data = {'keys': ["1330021122101132", "1330021122101133", "1330021122101311"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df37 = getinfo(jsonValueList)
display(stylize_simple(df37.head(4), '小平市'))

  0%|          | 0/8 [00:00<?, ?it/s]

,info,address,dt,cr
0,告知事項あり,東京都小平市小川東町五丁目一八丁目38-2 アルカンシエル・セブン 1階,不明,平成29年10月31日
1,205号室 心理的瑕疵あり,東京都小平市小川東町五丁目18-38-2 アルカンシエルセブン,令和4年10月17日,令和4年10月18日
2,奥様が自殺。今は取り壊されている,東京都小平市仲町628-64,20年くらい前,平成26年1月9日
3,まだ畑だった頃、灯油をかぶり、焼身自殺 。畳が重ねて置いてありましたが生々しく人の形に黒く焦げていました,東京都小平市小川東町二丁目1808,23年くらい前,平成26年1月9日


In [44]:
#* '羽村市'
data = {'keys': ["1330021120232020", "1330021120232022"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df38 = getinfo(jsonValueList)
display(stylize_simple(df38.head(4), '羽村市'))

  0%|          | 0/5 [00:00<?, ?it/s]

,info,address,dt,cr
0,室内お亡くなり,東京都羽村市栄町二丁目10-4 103,202102,令和3年3月10日
1,絞殺,東京都羽村市栄町二丁目6-4,令和5年6月18日,令和5年6月20日
2,一階角部屋自殺,東京都羽村市栄町二丁目11-8,2018年夏ごろ,平成30年10月8日
3,駐車スペース横の部屋で住民男性が首吊り自殺,東京都羽村市緑ヶ丘五丁目1-50,2005,令和4年4月9日


In [45]:
#* '武蔵村山市'
data = {'keys': ["1330021120233312", "1330021120233330"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df39 = getinfo(jsonValueList)
display(stylize_simple(df39.head(4), '武蔵村山市'))

  0%|          | 0/2 [00:00<?, ?it/s]

,info,address,dt,cr
0,御爺さん老衰し、その後御婆さん転居し転売される。,東京都武蔵村山市本町三丁目60-5,平成１２年頃,平成29年5月29日
1,死体発見,東京都武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,平成24年,平成27年1月18日


In [46]:
#* '昭島市'
data = {'keys': ["1330021122013000", "1330021122013002"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df40 = getinfo(jsonValueList)
display(stylize_simple(df40.head(4), '昭島市'))

  0%|          | 0/3 [00:00<?, ?it/s]

,info,address,dt,cr
0,家主が駐車場の車の中で服薬自殺,東京都昭島市緑町一丁目8,約10年前,令和2年3月16日
1,老婆と長女（60代）が暮らしていたが、 生活が困窮していて、長女が餓死したらしい、 老婆は生きていた。,東京都昭島市緑町一丁目9,2019/12/27,令和2年7月13日
2,火災による死亡,東京都昭島市田中町一丁目34-9,令和2年9月7日,令和2年9月7日


In [47]:
#* '福生市'
data = {'keys': ["1330021122010013", "1330021122010031"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df41 = getinfo(jsonValueList)
display(stylize_simple(df41.head(4), '福生市'))

  0%|          | 0/4 [00:00<?, ?it/s]

,info,address,dt,cr
0,5階 角部屋 南西向き2LDK（洋室5.6帖、洋室5.4帖、LDK14.2帖） 専有面積55.38㎡ 心理的瑕疵有り 定期借家（2年） リノベーション物件 、リフォーム物件,東京都福生市大字福生693-1 エアーシリウス,令和4年5月14日,令和4年5月17日
1,心理的瑕疵有り,東京都福生市志茂243-2 ＴＯＰ福生第２ 4階,不明,平成27年3月25日
2,無職の男性(26)方から出火し、部屋を半焼。男性は全身やけどなどで死亡した。 5階建てマンション「山本第一コーポ」5階503号室 1987年築 RC造 ＪＲ中央線/福生駅 徒歩7分,東京都福生市本町80-3,平成16年3月14日,平成23年9月21日
3,孤高死 腐乱死体の噂あり,東京都福生市北田園一丁目12-1,2015～2016頃,令和元年11月1日


## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#0B666A; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #0B666A">전체 46개 구역 중 4개 구역의 데이터는 존재하지 않는다.</p>

In [48]:
dataframeNameList = [f"df{idx}" for idx in range(1, len(jp_district_list) - 4)]
result = pd.DataFrame()
for df in dataframeNameList:
    result = pd.concat([result, eval(df)])

In [49]:
result.rename(columns = {'dt' : 'occurred_date', 'cr' : 'recorded_date'}, inplace = True)
result.reset_index(drop=True, inplace=True)

In [50]:
#* \n, \r 제거
def remove_newlines(text):
    return text.replace('\n', ' ').replace('\r', ' ')

In [51]:
#* 특수문자 제거
#* string.punctuation에 있는 특수문자를 모두 제거
#* string.punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
# 특수문자를 제거하는 함수 정의
def remove_special_characters(text):
    # string.punctuation에 있는 특수문자를 빈 문자열('')로 대체하여 제거
    for char in text:
        if char in string.punctuation:
            text = text.replace(char, '')
    return text
result = result.applymap(remove_special_characters)
display(stylize_simple(result.head(4), 'Total Dataset'))

,info,address,occurred_date,recorded_date
0,819号室で女性死亡,東京都新宿区歌舞伎町一丁目202,平成30年5月23日,平成30年7月4日
1,飛び降り自殺,東京都新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,令和3年5月18日,令和3年5月26日
2,告知事項あり ※管理会社に直接確認済み ハイム大成ビル 403号室 賃料¥78000 管理費¥2000 前入居者が室内で自殺,東京都新宿区西新宿七丁目1813,1年以内,令和3年2月23日
3,１４Ｆから女性が飛び降り自殺,東京都新宿区西新宿七丁目1713パレステュディオ新宿WEST,令和元年9月23日,令和元年9月23日


In [52]:
# #* 특수문자 제거2
# def remove_special_characters2(text):
#     return text.replace('(', '').replace(')', '')

# result = result.applymap(remove_special_characters2)
# result
#! (, ) 제거가 안됨
#! 화살표도 존재

In [53]:
result.to_csv('oshimaland_dataset.csv', index = False)